In [1]:
# Preamble 
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

from astropy.table import Table,join,hstack,vstack
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import corner
import time
import pickle
import imageio as iio
from datetime import date

In [2]:
dates = glob.glob('daily/*.fits')
dates = [date[38:38+6] for date in dates]
dates = np.unique(dates)
dates.sort()

# import os
# for p in ps:
#     os.system('ipython galah_dr4_post_processing.py '+p)
print(len(dates),list(dates))

505 ['131216', '131217', '131220', '140111', '140112', '140113', '140114', '140115', '140116', '140117', '140118', '140207', '140208', '140209', '140210', '140211', '140212', '140303', '140304', '140305', '140307', '140308', '140309', '140310', '140312', '140313', '140314', '140315', '140316', '140409', '140412', '140413', '140414', '140607', '140608', '140609', '140610', '140611', '140707', '140708', '140709', '140710', '140711', '140713', '140805', '140806', '140807', '140808', '140809', '140810', '140811', '140812', '140814', '140822', '140823', '140824', '141031', '141102', '141103', '141104', '141202', '141231', '150101', '150102', '150103', '150105', '150106', '150107', '150108', '150109', '150112', '150204', '150205', '150206', '150207', '150208', '150209', '150210', '150211', '150330', '150401', '150405', '150406', '150407', '150408', '150409', '150410', '150411', '150412', '150413', '150426', '150427', '150428', '150429', '150430', '150504', '150531', '150601', '150602', '1506

In [3]:
def combine_dates(dates):
    
    data = Table.read('daily/galah_dr4_allspec_not_validated_'+str(dates[0])+'_single.fits')
    data_next = Table.read('daily/galah_dr4_allspec_not_validated_'+str(dates[0])+'_coadds.fits')
    data = vstack([data, data_next])
    data_next = Table.read('daily/galah_dr4_allspec_not_validated_'+str(dates[0])+'_binary.fits')
    data = vstack([data, data_next])
    
    for setup in ['single','coadds','binary']:
        for date in dates[1:]:
            try:
                data_next = Table.read('daily/galah_dr4_allspec_not_validated_'+str(date)+'_'+setup+'.fits')
                data = vstack([data, data_next])
            except:
                pass

    return(data)

data = combine_dates(dates)

In [4]:
a_file = open("final_flag_sp_dictionary.pkl", "rb")
flag_sp_dictionary = pickle.load(a_file)
a_file.close()

binary_setup = data['setup']=='binary'

triple = binary_setup & ((data['flag_sp'] & flag_sp_dictionary['is_sb2'][0]) > 0)
double = binary_setup & ((data['flag_sp'] & flag_sp_dictionary['is_sb2'][0]) == 0)

data['flag_sp'][double] += flag_sp_dictionary['is_sb2'][0]

data['flag_sp'][triple] += flag_sp_dictionary['sb_triple_warn'][0]

In [5]:
# for element in [
#         'Li','C','N','O',
#         'Na','Mg','Al','Si',
#         'K','Ca','Sc','Ti','V','Cr','Mn','Co','Ni','Cu','Zn',
#         'Rb','Sr','Y','Zr','Mo','Ru',
#         'Ba','La','Ce','Nd','Sm','Eu'
# ]:
#     data[element.lower()+'_fe'][np.where(data['flag_'+element.lower()+'_fe'] == -1)[0]] = np.NaN
#     data['e_'+element.lower()+'_fe'][np.where(data['flag_'+element.lower()+'_fe'] == -1)[0]] = np.NaN
#     data[element.lower()+'_fe'][np.where(data['flag_'+element.lower()+'_fe'] == 2)[0]] = np.NaN
#     data['flag_'+element.lower()+'_fe'][np.where(data['flag_'+element.lower()+'_fe'] == -1)[0]] = 2
# data['fe_h'][np.where(data['flag_fe_h'] == -1)[0]] = np.NaN
# data['e_fe_h'][np.where(data['flag_fe_h'] == -1)[0]] = np.NaN
# data['fe_h'][np.where(data['flag_fe_h'] == 2)[0]] = np.NaN
# data['flag_fe_h'][np.where(data['flag_fe_h'] == -1)[0]] = 2

In [6]:
data[[0,1,2,3,4,-5,-4,-3,-2,-1]]

sobject_id,tmass_id,gaiadr3_source_id,survey_name,field_id,setup,mjd,ra,dec,best_spec4star,flag_sp,flag_sp_fit,flag_red,snr_px_ccd1,snr_px_ccd2,snr_px_ccd3,snr_px_ccd4,chi2_sp,px_used_perc,model_name,closest_model,comp_time,flux_contr,e_flux_contr,rv_comp_nr,rv_comp_1,e_rv_comp_1,rv_comp_1_h,rv_comp_1_p,rv_comp_2,e_rv_comp_2,rv_comp_2_h,rv_comp_2_p,rv_gaia_dr3,e_rv_gaia_dr3,v_bary_eff,teff,e_teff,logg,e_logg,fe_h,e_fe_h,flag_fe_h,vmic,e_vmic,vsini,e_vsini,teff_comp_2,e_teff_comp_2,logg_comp_2,e_logg_comp_2,fe_h_comp_2,e_fe_h_comp_2,flag_fe_h_comp_2,vmic_comp_2,e_vmic_comp_2,vsini_comp_2,e_vsini_comp_2,li_fe,e_li_fe,flag_li_fe,c_fe,e_c_fe,flag_c_fe,n_fe,e_n_fe,flag_n_fe,o_fe,e_o_fe,flag_o_fe,na_fe,e_na_fe,flag_na_fe,mg_fe,e_mg_fe,flag_mg_fe,al_fe,e_al_fe,flag_al_fe,si_fe,e_si_fe,flag_si_fe,k_fe,e_k_fe,flag_k_fe,ca_fe,e_ca_fe,flag_ca_fe,sc_fe,e_sc_fe,flag_sc_fe,ti_fe,e_ti_fe,flag_ti_fe,v_fe,e_v_fe,flag_v_fe,cr_fe,e_cr_fe,flag_cr_fe,mn_fe,e_mn_fe,flag_mn_fe,co_fe,e_co_fe,flag_co_fe,ni_fe,e_ni_fe,flag_ni_fe,cu_fe,e_cu_fe,flag_cu_fe,zn_fe,e_zn_fe,flag_zn_fe,rb_fe,e_rb_fe,flag_rb_fe,sr_fe,e_sr_fe,flag_sr_fe,y_fe,e_y_fe,flag_y_fe,zr_fe,e_zr_fe,flag_zr_fe,mo_fe,e_mo_fe,flag_mo_fe,ru_fe,e_ru_fe,flag_ru_fe,ba_fe,e_ba_fe,flag_ba_fe,la_fe,e_la_fe,flag_la_fe,ce_fe,e_ce_fe,flag_ce_fe,nd_fe,e_nd_fe,flag_nd_fe,sm_fe,e_sm_fe,flag_sm_fe,eu_fe,e_eu_fe,flag_eu_fe,sb2_rv_16,sb2_rv_50,sb2_rv_84,ew_h_beta,ew_h_alpha,ew_k_is,sigma_k_is,rv_k_is,ew_dib5780,sigma_dib5780,rv_dib5780,ew_dib5797,sigma_dib5797,rv_dib5797,ew_dib6613,sigma_dib6613,rv_dib6613
int64,bytes16,int64,bytes17,int64,bytes6,float32,float64,float64,bool,int64,int64,int64,float32,float32,float32,float32,float32,int64,bytes16,bytes16,float32,float32,float32,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32,float32,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
131216001101002,05190449-5849304,4762794963745841536,other,-1,single,56642.574,79.76874542236328,-58.82512664794922,False,0,0,0,24.843557,36.874645,52.64503,46.322086,0.78330594,94,5000_3.00_-0.50,5000_3.00_-0.50,67.70332,1.0,nan,1,5.943003,0.02787496,1.0,0.49527532,nan,nan,nan,nan,5.762518,2.2376232,-2.427048444747925,4928.9565,13.075967,3.1382346,0.03142668,-0.45227367,0.012714051,0,1.2481477,0.024025956,3.8052006,0.19405611,nan,nan,nan,nan,nan,nan,-1,nan,nan,nan,nan,0.08512212,0.03701036,0,0.22942676,0.032223526,0,0.10341218,0.046772733,1,0.61358416,0.078634985,0,0.17305218,0.03298461,0,0.33884394,0.020970011,0,0.55018526,0.05601975,0,0.23751633,0.019419441,0,0.2942443,0.0547015,0,0.21983638,0.025115373,0,0.26524568,0.025471244,0,0.37495726,0.01167009,0,0.39241278,0.015931519,0,0.010236704,0.01714068,0,-0.17235014,0.027856022,0,0.2637131,0.01929502,0,0.103581674,0.014190631,0,0.000264436,0.038977906,0,0.17401001,0.082138725,0,nan,nan,2,nan,nan,2,0.08384642,0.042818673,0,0.3009979,0.10843913,1,nan,nan,2,nan,nan,2,-0.18387114,0.05589318,0,0.43408802,0.036995072,1,-0.13416252,0.09846622,1,0.21102019,0.054051306,0,0.3544555,0.096751876,0,nan,nan,2,nan,nan,nan,0.17966375,0.

# Populate best_spec4star

In [ ]:
debug = False
# debug = True

for tmass_id in np.unique(data['tmass_id']):
    
    if tmass_id != 'None':

        # First let's see how many matches we actually have
        find_matches = np.where(tmass_id == data['tmass_id'])[0]

        # If only 1 entry found, we most likely can simply take that
        if len(find_matches) == 1:

            # Sanity check: is it really a single one?
            if data['setup'][find_matches[0]] == 'single':
                data['best_spec4star'][find_matches[0]] = True
            # If not: you better run the single setup as well!
            else:
                raise ValueError('1 entry found, but not single?!')

        # If we have multiple matches: check out which ones are single/binary/coadds
        else:

            single_matches = find_matches[np.where(data['setup'][find_matches]=='single')[0]]
            binary_matches = find_matches[np.where(data['setup'][find_matches]=='binary')[0]]
            coadds_matches = find_matches[np.where(data['setup'][find_matches]=='coadds')[0]]
            if len(coadds_matches) > 1:
                raise ValueError('Multiple coadded measurements available?!')
            elif len(coadds_matches) == 1:
                coadds_match = coadds_matches[0]
            nonbin_matches = find_matches[np.where(data['setup'][find_matches]!='binary')[0]]

            if debug:
                print(tmass_id,':',list(data['sobject_id'][find_matches]))
                print('    flag_sp:',list(find_matches),list(data['setup'][find_matches]))
                print('    flag_sp:',list(data['flag_sp'][find_matches]))

            # The rundown of our preference:
            # if it is clearly better fit with a binary analysis: prefer that!
            #     Double check if there are mutiple binary analyses and take 
            #       1) the highest unflagged one
            #       2) the unflagged one
            #       3) the highest snr one
            # elif there is a coadds run, prefer that
            # else: there should be a coadds run!

            # When do we believe it is a binary?
            it_is_a_binary = False

            # Let's test that, if binary setup(s) avaliable
            if len(binary_matches) > 0:

                if debug:
                    print('--> binary found!')

                # If only 1 binary measurement is available
                if len(binary_matches) == 1:
                    binary_match = binary_matches[0]

                    # chi2 has to be better by at least 5% in more than 50% of the cases
                    binary_criterion_a = (np.nanmedian(data['chi2_sp'][binary_match] / list(data['chi2_sp'][nonbin_matches])) < 0.95)
                    # ΔRV of the 2 binary components has to be beyond 10 km/s
                    binary_criterion_b = (np.abs(data['rv_comp_1'][binary_match] - data['rv_comp_2'][binary_match]) > 10)
                    # ΔRV of the 2 binary components has to be within 300 km/s
                    binary_criterion_c = (np.abs(data['rv_comp_1'][binary_match] - data['rv_comp_2'][binary_match]) < 300)

                    if (
                        binary_criterion_a &
                        binary_criterion_b &
                        binary_criterion_c
                    ):
                        it_is_a_binary = True

                    if debug:
                        print('    a) chi2 ratios:',binary_criterion_a,data['chi2_sp'][binary_match] / list(data['chi2_sp'][nonbin_matches]))
                        print('    delta rv1/2:',binary_criterion_b,data['rv_comp_1'][binary_match],data['rv_comp_2'][binary_match],np.abs(data['rv_comp_1'][binary_match] - data['rv_comp_2'][binary_match]))
                        print('    b) delta rv1/2  >  10?: ',binary_criterion_b)
                        print('    c) delta rv1/2  < 300?: ',binary_criterion_c)
                        print('    Vertict?:',it_is_a_binary)

                    if it_is_a_binary:
                        data['best_spec4star'][binary_match] = True

                # If multiple binary measurements are available
                else:

                    single_matches_to_binary_matches = []
                    for sobject_id in data['sobject_id'][binary_matches]:

                        single_matches_to_binary_matches.append(single_matches[np.where(data['sobject_id'][single_matches] == sobject_id)[0][0]])

                    if np.all(data['sobject_id'][single_matches_to_binary_matches] == data['sobject_id'][binary_matches]):

                        # Let's first check if these binaries hold up the basic tests from criterions a,b,c
                        valid_binary_matches = (
                            (abs(data['chi2_sp'][binary_matches]/data['chi2_sp'][single_matches_to_binary_matches]) < 0.95) &
                            (abs(data['rv_comp_1'][binary_matches]-data['rv_comp_2'][binary_matches]) > 10) &
                            (abs(data['rv_comp_1'][binary_matches]-data['rv_comp_2'][binary_matches]) < 300)
                        )
                        # If there is a coadds match, let's also check agains that measurement
                        if len(coadds_matches) == 1:
                            valid_binary_matches = (
                                (abs(data['chi2_sp'][binary_matches]/data['chi2_sp'][single_matches_to_binary_matches]) < 0.95) &
                                (abs(data['chi2_sp'][binary_matches]/data['chi2_sp'][coadds_match]) < 0.95) &
                                (abs(data['rv_comp_1'][binary_matches]-data['rv_comp_2'][binary_matches]) > 10) &
                                (abs(data['rv_comp_1'][binary_matches]-data['rv_comp_2'][binary_matches]) < 300)
                            )

                        # If we have at least 1 good measurement
                        if len(binary_matches[valid_binary_matches]) > 0:

                            it_is_a_binary = True

                            # Then find the one with the largest separation and use it as best_spec4star
                            larger_rv_separation = np.argmax(abs(data['rv_comp_1'][binary_matches[valid_binary_matches]]-data['rv_comp_2'][binary_matches[valid_binary_matches]]))
                            data['best_spec4star'][binary_matches[valid_binary_matches][larger_rv_separation]] = True
                    else:

                        raise ValueError('sobject_id of single and binary not in the same order...')

            if not it_is_a_binary:

                # If it is not a binary, prefer the coadds
                if len(coadds_matches) == 1:

                    if debug:
                        print('--> coadds found (but not binary)!')

                        print(tmass_id,':',list(data['sobject_id'][find_matches]))
                        print('    flag_sp:',list(find_matches),list(data['setup'][find_matches]))
                        print('    flag_sp:',list(data['flag_sp'][find_matches]))

                        print('    SNRs:')
                        print('         ',data['snr_px_ccd2'][coadds_match])
                        print('         ',list(data['snr_px_ccd2'][single_matches]))

                    data['best_spec4star'][coadds_match] = True

                # If there is no coadds: take the single measurement
                elif len(single_matches) == 1:
                    data['best_spec4star'][single_matches[0]] = True

        if len(np.where(data['best_spec4star'][find_matches] == True)[0]) != 1:

            # This should only be activated, if we have not run coadds for all stars!

            best_single_match = single_matches[np.argmin(data['flag_sp'][single_matches])]
            data['best_spec4star'][best_single_match] = True

no_tmass_id = data['tmass_id'] == 'None'
entries = np.arange(len(data['sobject_id']))

for object_index, sobject_id in enumerate(data['sobject_id'][no_tmass_id]):
        
    index = entries[no_tmass_id][object_index]
    same_coordinates = np.where((data['ra'][index]==data['ra']) & (data['dec'][index]==data['dec']))[0]

    single_matches = np.where(data['setup'][same_coordinates] == 'single')[0]
    binary_matches = np.where(data['setup'][same_coordinates] == 'binary')[0]
    coadds_matches = np.where(data['setup'][same_coordinates] == 'coadds')[0]
    nonbin_matches = np.where(data['setup'][same_coordinates] != 'binary')[0]
    
    already_best = np.where(data['best_spec4star'][same_coordinates]==True)[0]

    it_is_a_binary = False
    
    if len(same_coordinates) == 1:
        data['best_spec4star'][index] = True
        
    elif len(already_best) > 0:
        pass
    
    elif len(binary_matches) > 0:
        
        if len(binary_matches) == 1:
            binary_match = binary_matches[0]
        
            # chi2 has to be better by at least 5% in more than 50% of the cases
            binary_criterion_a = (np.nanmedian(data['chi2_sp'][same_coordinates[binary_match]] / list(data['chi2_sp'][same_coordinates[nonbin_matches]])) < 0.95)
            # ΔRV of the 2 binary components has to be beyond 10 km/s
            binary_criterion_b = (np.abs(data['rv_comp_1'][same_coordinates[binary_match]] - data['rv_comp_2'][same_coordinates[binary_match]]) > 10)
            # ΔRV of the 2 binary components has to be within 300 km/s
            binary_criterion_c = (np.abs(data['rv_comp_1'][same_coordinates[binary_match]] - data['rv_comp_2'][same_coordinates[binary_match]]) < 300)
            
            if debug:
                print('    a) chi2 ratios:',binary_criterion_a,data['chi2_sp'][same_coordinates[binary_match]] / list(data['chi2_sp'][same_coordinates[nonbin_matches]]))
                print('    delta rv1/2:',binary_criterion_b,data['rv_comp_1'][same_coordinates[binary_match]],data['rv_comp_2'][same_coordinates[binary_match]],np.abs(data['rv_comp_1'][same_coordinates[binary_match]] - data['rv_comp_2'][same_coordinates[binary_match]]))
                print('    b) delta rv1/2  >  10?: ',binary_criterion_b)
                print('    c) delta rv1/2  < 300?: ',binary_criterion_c)
                print('    Vertict?:',it_is_a_binary)

            if (
                binary_criterion_a &
                binary_criterion_b &
                binary_criterion_c
            ):
                it_is_a_binary = True
        else:
            
            single_matches_to_binary_matches = []
            for sobject_id in data['sobject_id'][same_coordinates[binary_matches]]:

                single_matches_to_binary_matches.append(single_matches[np.where(data['sobject_id'][same_coordinates][single_matches] == sobject_id)[0][0]])
                
            if np.all(data['sobject_id'][same_coordinates][single_matches_to_binary_matches] == data['sobject_id'][same_coordinates[binary_matches]]):

                # Let's first check if these binaries hold up the basic tests from criterions a,b,c
                valid_binary_matches = (
                    (abs(data['chi2_sp'][same_coordinates[binary_matches]]/data['chi2_sp'][same_coordinates][single_matches_to_binary_matches]) < 0.95) &
                    (abs(data['rv_comp_1'][same_coordinates[binary_matches]]-data['rv_comp_2'][same_coordinates[binary_matches]]) > 10) &
                    (abs(data['rv_comp_1'][same_coordinates[binary_matches]]-data['rv_comp_2'][same_coordinates[binary_matches]]) < 300)
                )
                # If there is a coadds match, let's also check agains that measurement
                if len(coadds_matches) == 1:
                    valid_binary_matches = (
                        (abs(data['chi2_sp'][same_coordinates[binary_matches]]/data['chi2_sp'][same_coordinates][single_matches_to_binary_matches]) < 0.95) &
                        (abs(data['chi2_sp'][same_coordinates[binary_matches]]/data['chi2_sp'][same_coordinates][coadds_match]) < 0.95) &
                        (abs(data['rv_comp_1'][same_coordinates[binary_matches]]-data['rv_comp_2'][same_coordinates[binary_matches]]) > 10) &
                        (abs(data['rv_comp_1'][same_coordinates[binary_matches]]-data['rv_comp_2'][same_coordinates[binary_matches]]) < 300)
                    )
                    
                # If we have at least 1 good measurement
                if len(binary_matches[valid_binary_matches]) > 0:

                    it_is_a_binary = True

                    # Then find the one with the largest separation and use it as best_spec4star
                    larger_rv_separation = np.argmax(abs(data['rv_comp_1'][same_coordinates[binary_matches[valid_binary_matches]]]-data['rv_comp_2'][same_coordinates[binary_matches[valid_binary_matches]]]))
                    data['best_spec4star'][same_coordinates][binary_matches[valid_binary_matches][larger_rv_separation]] = True
            else:

                raise ValueError('sobject_id of single and binary not in the same order...')

    if not it_is_a_binary:

        if len(coadds_matches) > 0:
            
            if len(coadds_matches) == 1:
                data['setup'][same_coordinates][coadds_matches[0]] = True
            
            else:
                raise ValueError('Multiple coadds measurements for this non-2MASS star')
        elif len(single_matches) == 1:
            data['best_spec4star'][same_coordinates][single_matches[0]] = True

        else:

            # This should only be activated, if we have not run coadds for all stars!

            if debug:
                print('There should be a coadds spectrum here too!')

            best_single_match = single_matches[np.argmin(data['flag_sp'][same_coordinates][single_matches])]
            data['best_spec4star'][same_coordinates[best_single_match]] = True


/pkg/linux/anaconda-20210218/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


# Save FITS Files

In [ ]:
data.write('galah_dr4_allspec_not_validated.fits',overwrite=True)
print(len(data['sobject_id']))

data_allstar = data[data['best_spec4star']==True]
data_allstar.sort(keys='ra')
data_allstar.write('galah_dr4_allstar_not_validated.fits',overwrite=True)
data_allstar

# Flag dictionaries

In [ ]:
a_file = open("final_flag_sp_dictionary.pkl", "rb")
flag_sp_dictionary = pickle.load(a_file)
a_file.close()
print(flag_sp_dictionary)

entries = []
for flag in np.unique(data['flag_sp']):
    
    flag_text = []
    for flag_key in flag_sp_dictionary.keys():
        if((flag & flag_sp_dictionary[flag_key][0]) == flag_sp_dictionary[flag_key][0]):
            flag_text.append(flag_key)
    entries.append([flag,len(data['flag_sp'][data['flag_sp']==flag]),", ".join(flag_text)])
entries = np.array(entries)

a = Table()
a['flag_sp'] = entries[:,0]
a['nr_spectra'] = entries[:,1]
a['flag_sp_keys'] = entries[:,2]
for s in flag_sp_dictionary.keys():
    print(flag_sp_dictionary[s][0],flag_sp_dictionary[s][1])

In [ ]:
bad = data[(data['flag_sp'] >= 64) & (data['flag_sp'] < np.max(data['flag_sp'])) & (data['teff'] > 6500)]
bad

In [ ]:
from scipy.spatial import cKDTree
grids = Table.read('../spectrum_grids/galah_dr4_model_trainingset_gridpoints.fits')
grid_index_tree = cKDTree(np.c_[grids['teff_subgrid'],grids['logg_subgrid'],grids['fe_h_subgrid']])

model_needed = []
for i in data[(data['flag_sp'] >= 64) & (data['flag_sp'] < np.max(data['flag_sp']))]:
    model_needed.append(grid_index_tree.query([i['teff'],i['logg'],i['fe_h']],k=1)[1])
model_needed = np.array(model_needed)
models_needed = Table()
models_needed['models'], models_needed['counts'] = np.unique(model_needed,return_counts=True)
models_needed.sort(keys = 'counts', reverse=True)
models_needed[:10]

In [ ]:
number = Table()
number['closest_model'],number['count'] = np.unique(data['closest_model'][(data['flag_sp_fit'] == 1)],return_counts=True)
number.sort(keys='count',reverse=True)
number[:10]

In [ ]:
# date = [str(x)[:6] for x in data['sobject_id'][(data['flag_sp_fit'] == 1) & (data['closest_model']=='5000_2.00_-0.75')]]
data['sobject_id'][(data['flag_sp_fit'] == 1) & (data['closest_model']=='5000_2.00_-0.75')]

In [ ]:
data[(data['n_fe'] > 1) & (data['flag_n_fe'] == 0) & (data['flag_sp'] == 0)][['sobject_id','model_name','closest_model','flag_sp','c_fe','flag_c_fe','n_fe','flag_n_fe','o_fe','flag_o_fe']]

In [ ]:
# Plot HRD & abundances 

flag_sp_0 = data['flag_sp'] == 0
flag_sp_above0_but_results = (data['flag_sp'] > 0) & (data['flag_sp'] < np.max(data['flag_sp']))
flag_sp_results = data['flag_sp'] < np.max(data['flag_sp'])

for label in [
    'Li',
    'C',
    'N',
    'O',
    'Na',
    'Mg',
    'Al',
    'Si',
    'K',
    'Ca',
    'Sc',
    'Ti',
    'V',
    'Cr',
    'Mn',
    'Co',
    'Ni',
    'Cu',
    'Zn',
    'Rb',
    'Sr',
    'Y',
    'Zr',
    'Mo',
    'Ru',
    'Ba',
    'La',
    'Ce',
    'Nd',
    'Sm',
    'Eu'
    ]:
    
    flag0 = flag_sp_0 & (data['flag_'+label.lower()+'_fe'] == 0) #& (data['fe_h'] > -1)
    flag1 = flag_sp_0 & (data['flag_'+label.lower()+'_fe'] == 1) #& (data['fe_h'] > -1)
    flag_rest = flag_sp_above0_but_results & (data['flag_'+label.lower()+'_fe'] <= 1)

    f, gs = plt.subplots(1,3,figsize=(10,3),sharey=True)

    xbins = np.linspace(-2.5,0.75,50)
    if label == 'Li':
        ybins = np.linspace(0,4,50)
    elif label in ['C','N','O','Y','Ba','La','Ce','Nd']:
        ybins = np.linspace(-1,2,50)
    elif label in ['Mg','Si','Ti']:
        ybins = np.linspace(-0.5,1,50)
    else:
        ybins = np.linspace(-1,1,50)
    
    if label == 'Li':
        ydata = data[label.lower()+'_fe'] + data['fe_h'] + 1.05
    else:
        ydata = data[label.lower()+'_fe']
    
    # First panel: Detections for GALAH DR4 [Fe/H] vs. [X/Fe]
    ax = gs[0]
    ax.text(0.05,0.9,'a) Detections for ['+label+'/Fe]',ha='left',transform=ax.transAxes,fontsize=12,bbox=dict(boxstyle='round', facecolor='w', alpha=0.75))
    ax.text(0.05,0.785,str(len(data['fe_h'][flag0]))+' ('+"{:.0f}".format(100*len(data['fe_h'][flag0])/len(data['fe_h'][flag_sp_results]))+r'%)',ha='left',transform=ax.transAxes,bbox=dict(boxstyle='round', facecolor='w', alpha=0.75,lw=0))
    ax.set_xlabel('[Fe/H] (GALAH DR4)')
    if label == 'Li':
        ax.set_ylabel('A('+label+') (GALAH DR4)')
    else:
        ax.set_ylabel('['+label+'/Fe] (GALAH DR4)')

    if len(data['fe_h'][flag0]) > 10:
        corner.hist2d(
            data['fe_h'][flag0],
            ydata[flag0],
            bins = (xbins,ybins),
            range=[(xbins[0],xbins[-1]),(ybins[0],ybins[-1])],
            ax = ax
        )
    ax.set_xlim(xbins[0],xbins[-1])
    ax.set_ylim(ybins[0],ybins[-1])
    ax.errorbar(
        0.9*xbins[0]+0.1*xbins[-1],
        0.9*ybins[0]+0.1*ybins[-1],
        xerr=np.ma.median(data['e_fe_h'][flag0]),
        yerr=np.ma.median(data['e_'+label.lower()+'_fe'][flag0]),
        capsize=2,color='k'
    )
    
    # Second panel: Upper Limits for GALAH DR4 [Fe/H] vs. [X/Fe]
    ax = gs[1]
    ax.text(0.05,0.9,'b) Upper limits',ha='left',transform=ax.transAxes,fontsize=12,bbox=dict(boxstyle='round', facecolor='w', alpha=0.75))
    ax.text(0.05,0.785,str(len(data['fe_h'][flag1]))+' ('+"{:.0f}".format(100*len(data['fe_h'][flag1])/len(data['fe_h'][flag_sp_results]))+r'%)',ha='left',transform=ax.transAxes,bbox=dict(boxstyle='round', facecolor='w', alpha=0.75,lw=0))
    ax.set_xlabel('[Fe/H] (GALAH DR4)')
    
    if len(data['fe_h'][flag1]) > 10:
        corner.hist2d(
            data['fe_h'][flag1],
            ydata[flag1],
            bins = (xbins,ybins),
            range=[(xbins[0],xbins[-1]),(ybins[0],ybins[-1])],
            ax = ax
        )
    ax.set_xlim(xbins[0],xbins[-1])
    ax.set_ylim(ybins[0],ybins[-1])
    ax.errorbar(
        0.9*xbins[0]+0.1*xbins[-1],
        0.9*ybins[0]+0.1*ybins[-1],
        xerr=np.ma.median(data['e_fe_h'][flag1]),
        yerr=np.ma.median(data['e_'+label.lower()+'_fe'][flag1]),
        capsize=2,color='k'
    )

    # Second panel: Upper Limits for GALAH DR4 [Fe/H] vs. [X/Fe]
    ax = gs[2]
    ax.text(0.05,0.9,'c) Flagged Spectra',ha='left',transform=ax.transAxes,fontsize=12,bbox=dict(boxstyle='round', facecolor='w', alpha=0.75))
    ax.text(0.05,0.785,str(len(data['fe_h'][flag_rest]))+' ('+"{:.0f}".format(100*len(data['fe_h'][flag_rest])/len(data['fe_h'][flag_sp_results]))+r'%)',ha='left',transform=ax.transAxes,bbox=dict(boxstyle='round', facecolor='w', alpha=0.75,lw=0))
    ax.set_xlabel('[Fe/H] (GALAH DR4)')

    if len(data['fe_h'][flag_rest]) > 10:
        corner.hist2d(
            data['fe_h'][flag_rest],
            ydata[flag_rest],
            bins = (xbins,ybins),
            range=[(xbins[0],xbins[-1]),(ybins[0],ybins[-1])],
            ax = ax
        )
    ax.set_xlim(xbins[0],xbins[-1])
    ax.set_ylim(ybins[0],ybins[-1])
    ax.errorbar(
        0.9*xbins[0]+0.1*xbins[-1],
        0.9*ybins[0]+0.1*ybins[-1],
        xerr=np.ma.median(data['e_fe_h'][flag_rest]),
        yerr=np.ma.median(data['e_'+label.lower()+'_fe'][flag_rest]),
        capsize=2,color='k'
    )
    
    plt.tight_layout()
    plt.show()
#     plt.savefig('figures/galah_dr4_validation_overview_'+label.lower()+'_fe_density.png',dpi=200,bbox_inches='tight')
    plt.close()

In [ ]:
def plot_galah_dr4_overview_A4(data,name):
    elements = [
        'Li',
        'C',
        'N',
        'O',
        'Na',
        'Mg',
        'Al',
        'Si',
        'K',
        'Ca',
        'Sc',
        'Ti',
        'V',
        'Cr',
        'Mn',
        'Co',
        'Ni',
        'Cu',
        'Zn',
        'Rb',
        'Sr',
        'Y',
        'Zr',
        'Mo',
        'Ru',
        'Ba',
        'La',
        'Ce',
        'Nd',
        'Sm',
        'Eu'
        ]

    fig, axs = plt.subplots(ncols=6, nrows=6, figsize=(11.75,8.25))
    gs = axs[0, 0].get_gridspec()
    for ax in axs[0, :2]:
        ax.remove()
    for ax in axs[1, :2]:
        ax.remove()

    # HRD 

    flag_sp_0 = data['flag_sp'] == 0
    flag_sp_above0_but_results = (data['flag_sp'] > 0) & (data['flag_sp'] < np.max(data['flag_sp']))
    flag_sp_results = data['flag_sp'] < np.max(data['flag_sp'])

    teff_limits = [3750,7250]
    logg_limits = [-0.25,5.5]

    axbig = fig.add_subplot(gs[:2, :2])
    corner.hist2d(
        data['teff'][flag_sp_0],
        data['logg'][flag_sp_0],
        bins=(np.linspace(teff_limits[0],teff_limits[1],50),np.linspace(logg_limits[0],logg_limits[1],50)),
        ax = axbig
    )
    axbig.set_xlim(teff_limits[1],teff_limits[0])
    axbig.set_ylim(logg_limits[1],logg_limits[0])
    if name == 'allspec':
        axbig.text(0.5,0.93,str(len(data['teff'][flag_sp_0]))+r' unflagged spectra (of '+str(len(data['teff']))+')',transform=axbig.transAxes,ha='center',bbox=dict(boxstyle='square,pad=0',lw=0, facecolor='w', alpha=0.75))
    if name == 'allstar':
        axbig.text(0.5,0.93,str(len(data['teff'][flag_sp_0]))+r' unflagged stars (of '+str(len(data['teff']))+')',transform=axbig.transAxes,ha='center',bbox=dict(boxstyle='square,pad=0',lw=0, facecolor='w', alpha=0.75))
    axbig.text(0.5,0.035,r'$T_\mathrm{eff}~/~\mathrm{K}$',transform=axbig.transAxes,ha='center',bbox=dict(boxstyle='square,pad=0',lw=0, facecolor='w', alpha=0.75))
    axbig.text(0.02,0.5,r'$\log (g~/~\mathrm{cm\,s^{-2}})$',transform=axbig.transAxes,va='center',rotation=90,bbox=dict(boxstyle='square,pad=0',lw=0, facecolor='w', alpha=0.75))

    # read an image
    img = iio.imread("figures/logo_desktop.png")

    ax = axs[-1,-1]
    ax.imshow(img)
    ax.axis('off')
    ax.text(0.5,+1.2,'DR4 Preview',transform=ax.transAxes,ha='center')
    ax.text(0.6,-0.6,'Buder et al. \n as of '+date.today().strftime("%y%m%d"),transform=ax.transAxes,ha='center')

    for ind, label in enumerate(elements):    
        if ind <= 3:
            ax = axs[0,2+ind]
        elif ind <= 7:
            ax = axs[1,2+ind-4]
        elif ind <= 13:
            ax = axs[2,ind-8]
        elif ind <= 19:
            ax = axs[3,ind-14]
        elif ind <= 25:
            ax = axs[4,ind-20]
        elif ind <= 32:
            ax = axs[5,ind-26]
        else:
            raise ValueError('blob')
        if label == 'Li':
            color = '#2B292C'
        elif label in ['C','O','Na','Mg','Al','Si','K','Ca','Sc','Ti','Cu','Zn','Rb']:
            color = '#9AA2C9'
        elif label in ['V','Cr','Mn','Fe','Co','Ni']:
            color = '#E69774'
        elif label in ['N','Sr','Y','Zr','Mo','Ru','Ba','La','Ce','Nd','Sm']:
            color = '#C4DAAB'
        elif label in ['Eu']:
            color = '#D0A8C5'

        xbins = np.linspace(-2.5,0.75,50)
        if label == 'Li':
            ybins = np.linspace(-1,4,50)
        elif label in ['C','N','O','Y','Ba','La','Ce','Nd']:
            ybins = np.linspace(-1,2,50)
        elif label in ['Mg','Si','Ti']:
            ybins = np.linspace(-0.5,1,50)
        else:
            ybins = np.linspace(-1,1,50)

        # First panel: Detections for GALAH DR4 [Fe/H] vs. [X/Fe]
        if label == 'Li':
            textcolor = 'w'
            ax.set_yticks([-2,0,2])
        else:
            textcolor = 'k'

        flag0 = flag_sp_0 & (data['flag_'+label.lower()+'_fe'] == 0) #& (data['fe_h'] > -1)

        if label == 'Li':
            ax.text(0.05,0.83,'A('+label+')',ha='left',color=textcolor, transform=ax.transAxes,fontsize=10,bbox=dict(boxstyle='round', facecolor=color, alpha=0.75))
        else:
            ax.text(0.05,0.83,'['+label+'/Fe]',ha='left',color=textcolor, transform=ax.transAxes,fontsize=10,bbox=dict(boxstyle='round', facecolor=color, alpha=0.75))
        ax.text(0.97,0.05,"{:.0f}".format(100*len(data['e_fe_h'][flag0])/len(data['teff'][flag_sp_0]))+'%',ha='right', transform=ax.transAxes,fontsize=10,bbox=dict(boxstyle='square,pad=0',lw=0,facecolor='w', alpha=0.75))

        levels = (0.97,0.9,0.68,0.5,0.3)
        if label in ['La','Ce','Sm','Eu']:
            levels = (0.9,0.68,0.5,0.3)
        elif label in ['Rb','Sr','Mo','Ru']:
            levels = (0.5,0.3)
            
        ydata = data[label.lower()+'_fe'][flag0]
        if label == 'Li':
            ydata = data[label.lower()+'_fe'][flag0] + data['fe_h'][flag0] + 1.05
            
        #if (len(data['fe_h'][flag0]) > 20) & (label != 'Rb'):
        corner.hist2d(
            data['fe_h'][flag0],
            ydata,
            bins = (xbins,ybins),
            levels = levels,
            contour_kwargs=dict(colors=['k']),
            color=color,
            range=[(xbins[0],xbins[-1]),(ybins[0],ybins[-1])],
            ax = ax
        )
        ax.set_xlim(xbins[0],xbins[-1])
        ax.set_ylim(ybins[0],ybins[-1])
        ax.errorbar(
            0.9*xbins[0]+0.1*xbins[-1],
            0.9*ybins[0]+0.1*ybins[-1],
            xerr=np.ma.median(data['e_fe_h'][flag0]),
            yerr=np.ma.median(data['e_'+label.lower()+'_fe'][flag0]),
            capsize=2,color='k'
        )
        if ind >= 26:
            ax.set_xlabel('[Fe/H]')
    #     else:
        ax.set_xticks([-2,-1,0])
    plt.tight_layout(h_pad=0,w_pad=0)
    plt.savefig('figures/galah_dr4_overview_'+name+'.png',dpi=150,bbox_inches='tight')
    plt.show()
    plt.close()

plot_galah_dr4_overview_A4(data=data,name='allspec')
plot_galah_dr4_overview_A4(data=data_allstar,name='allstar')

In [ ]:
# Plot CNO

f, gs = plt.subplots(1,3,sharex=True,sharey=True,figsize=(2.5*3,2.5))

x_low  = -2.50
x_high =  0.75
y_low  = -1.00
y_high =  1.75

panels = ['a)','b)','c)','d)']

for i,label in enumerate(['C','N','O']):
#for i,label in enumerate(['C','N','CN','O']):
    ax = gs[i]

    if label in ['C','N','O']:
        flag0 = (data['flag_'+label.lower()+'_fe'] == 0) #& (data['fe_h'] > -1)
        flag1 = (data['flag_'+label.lower()+'_fe'] == 1) #& (data['fe_h'] > -1)
        corner.hist2d(
            data['fe_h'][flag0],
            data[label.lower()+'_fe'][flag0],
            ax = ax,bins=(np.linspace(x_low,x_high,50),np.linspace(y_low,y_high,50))
        )
#         ax.hist2d(
#             data['fe_h'][flag0],
#             data[label.lower()+'_fe'][flag0],
#             cmin = 1, bins=50
#             #s=1,label='Detection'
#         )
#         ax.scatter(
#             data['fe_h'][flag1],
#             data[label.lower()+'_fe'][flag1],
#             marker='v',label='Upper limit',
#             s=0.5
#         )
        
    if label == 'CN':
        flag0 = (data['flag_c_fe'] == 0) & (data['flag_n_fe'] == 0) #& (data['fe_h'] > -1)
        flag1 = ((data['flag_c_fe'] == 1) | (data['flag_n_fe'] == 1)) #& (data['fe_h'] > -1)
        corner.hist2d(
            data['fe_h'][flag0],
            data['c_fe'][flag0]-data['n_fe'][flag0],
            ax = ax,bins=(np.linspace(x_low,x_high,50),np.linspace(y_low,y_high,50))
        )
        #         ax.hist2d(
#             data['fe_h'][flag0],
#             data['c_fe'][flag0]-data['n_fe'][flag0],
#             cmin = 1, bins=50
# #             s=1,label='Detection'
#         )
#         ax.scatter(
#             data['fe_h'][flag1],
#             data['c_fe'][flag1]-data['n_fe'][flag1],
#             marker='v',label='Upper limit',
#             s=0.5
#         )
    
    ax.set_xlim(x_low,x_high)
    ax.set_ylim(y_low,y_high)
    
#     if i==0:
#         ax.legend()
    ax.set_xlabel('[Fe/H]')
    if label in ['C','N','O']:
        ax.set_ylabel('['+label+'/Fe]')
    if label == 'CN':
        ax.set_ylabel('[C/N]')
plt.tight_layout()
#plt.savefig('figures/overview_CNO_incl_upper_limits.png',dpi=200,bbox_inches='tight')
plt.savefig('figures/overview_CNO.png',dpi=200,bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# # [X/Fe] = [X/H] - [M/H]
# # [X/H] + (A_X - 12) = log(N_X/N_H) 
# # [X/Fe] = log(N_X / N_H) - log(N_X / N_H)_Sun - [Fe/H]
# # [C + N / Fe] = log((N_C + N_N) / N_H) - log((N_C_Sun + (N_N_Sun / N_H)_Sun - [Fe/H]
# # [C + N / Fe] = log(N_C/N_H + N_N/N_H) - log(N_C_Sun/H_Sun + N_N_Sun/N_H_Sun) - [Fe/H]

# A_N_Sun = 7.78+0.15
# A_C_Sun = 8.39+0.037

# A_C = data['c_fe'] + data['fe_h'] + (A_C_Sun)
# A_N = data['n_fe'] + data['fe_h'] + (A_N_Sun)

# N_C_N_H = 10**(A_C + 12)
# N_N_N_H = 10**(A_N + 12)
# N_C_N_H_sun = 10**(A_C_Sun + 12)
# N_N_N_H_sun = 10**(A_N_Sun + 12)

# data['cn_fe'] = np.log10(N_C_N_H + N_N_N_H) - np.log10(N_C_N_H_sun + N_N_N_H_sun) - data['fe_h']

In [ ]:
# def cn_masses(fe_h, c_fe, n_fe, cn_fe):
#     return(
#         1.08 - 0.18 * fe_h + 4.30 * c_fe +1.43 * n_fe - 7.55 * cn_fe
#         - 1.05 * (fe_h)**2 - 1.12 * (fe_h * c_fe) - 0.67 * (fe_h * n_fe) - 1.30 * (fe_h * cn_fe)
#         - 49.92 * (c_fe)**2 - 41.04 * (c_fe * n_fe) + 139.92 * (c_fe * cn_fe)
#         - 0.63 * (n_fe)**2 + 47.33 * (n_fe * cn_fe)
#         - 86.62 * (cn_fe)**2
#     )
# data['mass'] = cn_masses(fe_h=data['fe_h'], c_fe=data['c_fe'], n_fe=data['n_fe'], cn_fe=(data['cn_fe']-0.1)/2.)

In [ ]:
# def cn_ages(fe_h, c_fe, n_fe, cn_fe, teff, logg):
#     return(
#         -54.35 + 6.53*fe_h -19.02 *c_fe -12.18*n_fe +37.22*cn_fe +59.58*teff +16.14*logg
#         +0.74*fe_h*fe_h +4.04*fe_h*c_fe +0.76*fe_h*n_fe -4.94*fe_h*cn_fe -1.46*fe_h*teff -1.56*fe_h*logg
#         +26.90*c_fe*c_fe +13.33*c_fe*n_fe -77.84*c_fe*cn_fe +48.29*c_fe*teff -13.12*c_fe*logg
#         -1.04*n_fe*n_fe -17.60*n_fe*cn_fe +13.99*n_fe*teff -1.77*n_fe*logg
#         +51.24*cn_fe*cn_fe -65.67*cn_fe*teff +14.24*cn_fe*logg
#         +15.54*teff*teff -34.68*teff*logg
#         +4.17*logg*logg
#     )
# data['age'] = 10**(cn_ages(fe_h=data['fe_h'], c_fe=data['c_fe'], n_fe=data['n_fe'], cn_fe=data['cn_fe'], teff=data['teff']/4000., logg=data['logg']))

In [ ]:
# def hist2d_bin_colored(X,Y,Z,X_label='X\_label',Y_label='Y\_label',Z_label='Z\_label',bins=30,bin_function='median',ax=None,cmap='seismic_r',minimum_bin_entries = 5,**kwargs):
#     """
#     INPUT:
#     X : x-axis parameter
#     Y : y-axis parameter
#     Z : parameter that will be used for coloring the bins
#     X/Y/Z_label : label names
#     bins = 30, but you can also give it bins = (np.linspace(x_min,x_max,30),np.linspace(y_min,y_max,30))
#     bin_function : median/average/sum
#     ax : if you plot it as part of an f,ax = plt.subplots()
#     minimum_bin_entries : how many entries do we expect, before we even consider throwing some fancy function at them
    
#     OUTPUT:
#     plt.imshow
#     """
    
#     # First make sure we only work with finite values
#     finite = np.isfinite(X) & np.isfinite(Y) & np.isfinite(Z)
#     if len(X[finite])!=len(X):
#         print('Not all values were finite! Continuing with only finite ones')
#     X=X[finite];Y=Y[finite];Z=Z[finite]
    
#     # Now create the matrix of bins and its bin-edges
#     H,xedges,yedges = np.histogram2d(X,Y,bins=bins)

#     # Create the matrix that we want to store color-values in
#     color_matrix = np.zeros_like(H)
#     color_matrix[:] = np.nan
    
#     # Loop through the x- and y-bins
#     for x_bin in range(len(xedges)-1):
#         for y_bin in range(len(yedges)-1):
#             in_xy_bin = (X>=xedges[x_bin])&(X<xedges[x_bin+1])&(Y>=yedges[y_bin])&(Y<yedges[y_bin+1])
            
#             # We only add a value if there are more than *minimum_bin_entries* in the bin
#             if len(Z[in_xy_bin]) >= minimum_bin_entries:
#                 if bin_function=='median':
#                     color_matrix[x_bin,y_bin]=np.median(Z[in_xy_bin])
#                 elif bin_function=='average':
#                     color_matrix[x_bin,y_bin]=np.average(Z[in_xy_bin])
#                 elif bin_function=='sum':
#                     color_matrix[x_bin,y_bin]=np.sum(Z[in_xy_bin])
#                 elif bin_function=='std':
#                     color_matrix[x_bin,y_bin]=np.std(Z[in_xy_bin])
#                 else:
#                     raise NameError('Only bin_function = median/average/sum available')

#     # Create an axis if not given
#     if ax==None:
#         ax = plt.gca()
#     else:
#         ax=ax
#     ax.set_xlabel(X_label)
#     ax.set_ylabel(Y_label)

#     # Populate the keyword arguments for the imshow
#     imshow_kwargs = dict(
#         cmap = cmap,aspect='auto',origin='lower'
#     )
#     # Update by any arguments given through **kwargs
#     imshow_kwargs.update(kwargs)

#     # Plot!
#     s = ax.imshow(color_matrix.T,extent=(xedges[0],xedges[-1],yedges[0],yedges[-1]),**imshow_kwargs)
#     c = plt.colorbar(s, ax=ax)
#     c.set_label(Z_label)

In [ ]:
# giants = (data['logg'] < 3.5) & (data['flag_c_fe'] == 0) & (data['flag_n_fe'] == 0)

# f, gs = plt.subplots(2,2,figsize=(7,5))
# hist2d_bin_colored(
#     data['fe_h'][giants],
#     data['mg_fe'][giants],
#     data['mass'][giants],
#     X_label='[Fe/H]',
#     Y_label='[Mg/Fe]',
#     Z_label=r'Mass / $\mathrm{M}_\odot$',
#     ax = gs[0,0],
#     bins = (np.linspace(-0.8,0.3,30),np.linspace(-0.1,0.4,30)),
#     vmin = 0.8, vmax = 2.0,
#     cmap = 'RdYlBu_r',
#     minimum_bin_entries = 1
# )
# hist2d_bin_colored(
#     data['fe_h'][giants],
#     data['c_fe'][giants]-data['n_fe'][giants],
#     data['mass'][giants],
#     X_label='[Fe/H]',
#     Y_label='[C/N]',
#     Z_label=r'Mass / $\mathrm{M}_\odot$',
#     ax = gs[0,1],
#     bins = (np.linspace(-0.8,0.3,30),np.linspace(-0.8,0.4,30)),
#     vmin = 0.8, vmax = 2.0,
#     cmap = 'RdYlBu_r',
#     minimum_bin_entries = 1
# )
# hist2d_bin_colored(
#     data['teff'][giants],
#     data['logg'][giants],
#     data['mass'][giants],
#     X_label='Teff / K',
#     Y_label='logg',
#     Z_label=r'Mass / $\mathrm{M}_\odot$',
#     ax = gs[1,0],
#     bins = (np.linspace(3000,5500,30),np.linspace(0,4,30)),
#     vmin = 0.8, vmax = 2.0,
#     cmap = 'RdYlBu_r',
#     minimum_bin_entries = 1
# )
# gs[1,0].set_xlim(5500,3750)
# gs[1,0].set_ylim(4,0)
# hist2d_bin_colored(
#     data['c_fe'][giants]-data['n_fe'][giants],
#     data['mass'][giants],
#     data['fe_h'][giants],
#     X_label='[C/N]',
#     Y_label=r'Mass / $\mathrm{M}_\odot$',
#     Z_label='[Fe/H]',
#     ax = gs[1,1],
#     bins = (np.linspace(-0.8,0.4,30),np.linspace(0.5,3.0,30)),
#     vmin = -0.8, vmax = 0.3,
#     cmap = 'RdYlBu_r',
#     minimum_bin_entries = 1
# )
# plt.tight_layout()

In [ ]:
# giants = (data['logg'] > 1.8) & (data['logg'] < 3.3) & (data['flag_c_fe'] == 0) & (data['flag_n_fe'] == 0)

# f, gs = plt.subplots(2,2,figsize=(7,5))
# hist2d_bin_colored(
#     data['fe_h'][giants],
#     data['mg_fe'][giants],
#     data['age'][giants],
#     X_label='[Fe/H]',
#     Y_label='[Mg/Fe]',
#     Z_label=r'Age / Gyr',
#     ax = gs[0,0],
#     bins = (np.linspace(-0.8,0.3,30),np.linspace(-0.1,0.4,30)),
#     vmin = 0, vmax = 14,
#     cmap = 'RdYlBu_r',
#     minimum_bin_entries = 1
# )
# hist2d_bin_colored(
#     data['fe_h'][giants],
#     data['c_fe'][giants]-data['n_fe'][giants],
#     data['age'][giants],
#     X_label='[Fe/H]',
#     Y_label='[C/N]',
#     Z_label=r'Age / Gyr',
#     ax = gs[0,1],
#     bins = (np.linspace(-0.8,0.3,30),np.linspace(-0.8,0.4,30)),
#     vmin = 0, vmax = 14,
#     cmap = 'RdYlBu_r',
#     minimum_bin_entries = 1
# )
# hist2d_bin_colored(
#     data['teff'][giants],
#     data['logg'][giants],
#     data['age'][giants],
#     X_label='Teff / K',
#     Y_label='logg',
#     Z_label=r'Age / Gyr',
#     ax = gs[1,0],
#     bins = (np.linspace(3000,5500,30),np.linspace(0,4,30)),
#     vmin = 0, vmax = 14,
#     cmap = 'RdYlBu_r',
#     minimum_bin_entries = 1
# )
# gs[1,0].set_xlim(5500,3750)
# gs[1,0].set_ylim(4,0)
# hist2d_bin_colored(
#     data['c_fe'][giants]-data['n_fe'][giants],
#     data['age'][giants],
#     data['fe_h'][giants],
#     X_label='[C/N]',
#     Y_label=r'Age / Gyr',
#     Z_label='[Fe/H]',
#     ax = gs[1,1],
#     bins = (np.linspace(-0.8,0.4,30),np.linspace(0,14,30)),
#     vmin = -0.8, vmax = 0.3,
#     cmap = 'RdYlBu_r',
#     minimum_bin_entries = 1
# )
# plt.tight_layout()